after made some labels using lenslet, we can extract the labels and use them to train a model:

In [1]:
from pathlib import Path
import json

rated_dirs = [
    "/home/ubuntu/dev/lenslet/lenslet-backend/data/distill_dit_hires_user-prompt_bestof16-m1-aes12/images/batch_0",
    "/home/ubuntu/dev/lenslet/lenslet-backend/data/distill_dit_hires_user-prompt_bestof16-m1-aes12/images/batch_1",
]

# Collect all .png.json metadata files
all_metas = [str(p) for d in rated_dirs for p in Path(d).glob('*.png.json')]

# Load all metadata
meta_contents = []
for path in all_metas:
    with open(path, 'r', encoding='utf-8') as fh:
        meta_contents.append(json.load(fh))
path_meta_dict = dict(zip(all_metas, meta_contents))

# Filter only those with star == 5
filtered = {p: m for p, m in path_meta_dict.items() if m.get('star') == 5}
print(f"Filtered {len(filtered)} items with star == 5")

# Build list of image and JSON paths to upload
json_paths = list(filtered.keys())
img_paths = [p.removesuffix('.json') for p in json_paths]  # works for .png.json
all_paths = img_paths + json_paths
print(f"Prepared {len(all_paths)} files for upload")


Listing local files: 0files [00:00, ?files/s]

Listing local files: 0files [00:00, ?files/s]

Loading concurrent:   0%|          | 0/4000 [00:00<?, ?it/s]

{'type': 'list',
 'length': 210,
 'head': ['/home/ubuntu/dev/lenslet/lenslet-backend/data/distill_dit_hires_user-prompt_bestof16-m1-aes12/images/batch_0/948.png',
  '/home/ubuntu/dev/lenslet/lenslet-backend/data/distill_dit_hires_user-prompt_bestof16-m1-aes12/images/batch_0/743.png',
  '/home/ubuntu/dev/lenslet/lenslet-backend/data/distill_dit_hires_user-prompt_bestof16-m1-aes12/images/batch_0/938.png']}

In [ ]:
cp_commands = [f'aws s3 cp {p} s3://dataset-ingested/temp/lenslet-rated-5stars/' for p in all_paths]

import os
for cmd in cp_commands:
    os.system(cmd)

testing actual endpoint:

In [ ]:
import unibox as ub

imgs = ub.ls("s3://bucket-public-access-uw2/labelling2/export_aiimg_goods_batch1/zeka-old-5star/", ub.IMG_FILES)

In [4]:
import unibox as ub

imgs = ub.ls("s3://bucket-public-access-uw2/labelling2/export_aiimg_goods_batch1/zeka-old-5star/", ub.IMG_FILES)


import pandas as pd
import lenslet

# Example: DataFrame with mixed local and S3 images
df = pd.DataFrame({
    'image_path': imgs,
    # 'label': ['cat', 'dog', 'cat']
})

# Launch with multiple datasets
datasets = {
    "all_images": df['image_path'].tolist(),
    # "cats_only": df[df['label'] == 'cat']['image_path'].tolist(),
}

lenslet.launch(datasets, blocking=True, port=7071, verbose=True)



┌─────────────────────────────────────────────────┐
│                   🔍 Lenslet                    │
│         Lightweight Image Gallery Server        │
├─────────────────────────────────────────────────┤
│  Datasets:  all_images                          │
│  Images:    190                                 │
│  Server:    http://127.0.0.1:7071                     │
│  Mode:      In-memory (programmatic API)        │
└─────────────────────────────────────────────────┘



RuntimeError: asyncio.run() cannot be called from a running event loop